In [107]:
import sys, qiskit
sys.path.insert(0, '..')
import matplotlib.pyplot as plt
import numpy as np
import qiskit.quantum_info as qi
from qsee.compilation.qsp import QuantumStatePreparation
from qsee.core import ansatz, state, measure
from qsee.backend import constant, utilities
from qsee.evolution import crossover, mutate, selection, threshold
from qsee.evolution.environment import EEnvironment, EEnvironmentMetadata
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<img src = '../docs/compilation-design1.png' height ='500px'>
<img src = '../docs/compilation-design2.png' height ='500px'>

In [87]:
n = 10
m = 5
num_qubits = 2
utrains, utests = [], []
print("Training states:")
for i in range(0, n):
    utrain = state.haar(num_qubits)
    print(qi.Statevector.from_instruction(utrain).data)
    utrains.append(utrain)
print("Testing states:")
for i in range(0, m):
    utest = state.haar(num_qubits)
    print(qi.Statevector.from_instruction(utest).data)
    utests.append(utest)
    


def random_compilation_fitness(qc: qiskit.QuantumCircuit):
    p0s = []
    for i in range(0, n):
        qsp = QuantumStatePreparation(
            u=qc,
            target_state=utrains[i].inverse()
        ).fit(num_steps=10, metrics_func=['loss_basic'])
        # loss_basic = 1 - p0 => p0 = 1 - loss_basic
        p0s.append(1-qsp.compiler.metrics['loss_basic'][-1])
    # C = 1/n * sum(p_0)
    return np.mean(p0s)

def full_random_compilation_fitness(qc: qiskit.QuantumCircuit):
    p0s = []
    for i in range(0, n):
        qsp = QuantumStatePreparation(
            u=qc,
            target_state=utrains[i].inverse()
        ).fit(num_steps=100, metrics_func=['loss_basic'])
        # loss_basic = 1 - p0 => p0 = 1 - loss_basic
        p0s.append(1-qsp.compiler.metrics['loss_basic'][-1])
    # C = 1/n * sum(p_0)
    return np.mean(p0s)

def random_compiltion_test(qc_best: qiskit.QuantumCircuit):
    risks = []
    for i in range(0, m):
        qsp = QuantumStatePreparation(
            u=qc_best,
            target_state=utests[i].inverse()
        ).fit(num_steps=10)
        # loss_basic = 1 - p0 => p0 = 1 - loss_basic
        risks.append(measure.measure(utests[i]) - measure.measure(qc_best.assign_parameters(qsp.compiler.parameters)))
    # C = 1/n * sum(p_0)
    return np.mean(p0s)

Training states:
[0.76627381+0.j 0.30370054+0.j 0.35983787+0.j 0.43715802+0.j]
[-0.48642165+1.51643250e-16j  0.61234353-9.76577475e-19j
  0.59890639-1.82605912e-18j  0.17245437+2.47948546e-18j]
[ 0.06857528+1.18951136e-17j -0.73950579+8.11557423e-17j
 -0.62458783+8.93743665e-17j -0.24149256+4.27756235e-17j]
[ 0.65740703-1.85978918e-17j  0.45226981-9.09558233e-18j
  0.59621583+2.09007129e-18j -0.08828758+2.81867189e-17j]
[ 0.66656812+3.62847231e-34j -0.10554626+1.29256894e-17j
 -0.58981233+7.22311783e-17j  0.44347305-7.98033068e-34j]
[ 0.67236693+6.58798155e-18j  0.16914797-1.76459122e-18j
  0.0958498 -3.39035056e-18j -0.71423   +3.00168324e-16j]
[ 0.55497115-4.48327373e-18j  0.35778696+2.29068596e-17j
  0.58038686+7.19437822e-17j -0.476599  +1.12254628e-16j]
[ 0.10379983+2.46190708e-33j -0.51273235+6.27916027e-17j
  0.79678854-7.21993514e-34j -0.30242216+3.70360332e-17j]
[ 0.18234408-2.91534355e-19j -0.84223419+3.23643791e-16j
  0.40229887+4.32392310e-17j  0.3091081 +7.03222358e-18j]
[

In [114]:

# qubit = 2, depth = 4
# qubit = 3, depth around 15
# qubit = 4, depth around 40
# qubit = 5, depth around 100
# num_generation = 10, 20, 30, ...
# num_circuit = 4, 8, 16, 32, ...
# depth = 2,3,4, ...

def super_evol(_depth, _num_circuit, _num_generation):
    env_metadata = EEnvironmentMetadata(
        num_qubits = num_qubits,
        depth = _depth,
        num_circuit = _num_circuit,
        num_generation = _num_generation,
        prob_mutate=3/(_depth * _num_circuit)
    )
    env = EEnvironment(
        metadata = env_metadata,
        fitness_func=[random_compilation_fitness, full_random_compilation_fitness],
        selection_func=selection.elitist_selection,
        crossover_func=crossover.onepoint_crossover,
        mutate_func=mutate.layerflip_mutate,
        threshold_func=threshold.compilation_threshold
    )
    env.set_filename(f'n={num_qubits},d={_depth},n_circuit={_num_circuit},n_gen={_num_generation}')
    env.evol()

depths = [2,3]
num_circuit = 4
num_generations = [10, 20, 30]

for num_generation in num_generations: 
    for depth in depths:
        # check if folder exists
        import os
        if os.path.isdir(f'n={num_qubits},d={depth},n_circuit={num_circuit},n_gen={num_generation}') == False:
            print(depth, num_circuit, num_generation)
            super_evol(depth, num_circuit, num_generation)


In [111]:
env2 = EEnvironment.load('n=2,d=2,n_circuit=4,n_gen=2', random_compilation_fitness)
print(random_compilation_fitness(env2.best_circuits[0]))

0.552211017724849
